# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Bidding

In [2]:
vuln_ns, vuln_ew = False, False
hand = 'AQJ986.QJ952.7.8'

auction = ["PASS", "1N"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '2D', 'insta_score': 0.311, 'expected_score': 2, 'adjustment': 16},
 {'call': '2C', 'insta_score': 0.666, 'expected_score': -36, 'adjustment': 33}]

In [3]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


Tx.xx.T9xx.KJxxx xx.AKx.Q8xx.AQ9x AQJ98x.QJ9xx.x.8 Kxx.T8x.AKJx.Txx 0.99703 
 P-1N-2C-X-2H-P-P-X-P-P-P (5.0)  
 P-1N-2D-X-2H-X-P-3N-P-P-P (10.0) 
xxx.ATxx.Jxx.Qxx Txx.Kx.AKQx.KTxx AQJ98x.QJ9xx.x.8 K.8x.T98xx.AJ9xx 0.99823 
 P-1N-2C-X-2H-X-P-P-P (9.7)  
 P-1N-2D-X-2H-X-P-P-P (9.7) 
x.xx.J8xxx.KQJxx Kx.A8xx.KQT.A9xx AQJ98x.QJ9xx.x.8 Txxx.KT.A9xx.Txx 0.99914 
 P-1N-2C-X-2H-X-P-P-P (6.4)  
 P-1N-2D-X-2H-X-P-P-P (6.4) 
Txxx.8x.ATxx.Jxx Kx.Kxx.KQ8x.KQ9x AQJ98x.QJ9xx.x.8 x.ATx.J9xx.ATxxx 0.99826 
 P-1N-2C-X-2S-P-P-3C-P-P-P (9.5)  
 P-1N-2D-X-2S-X-P-P-P (10.0) 
Tx.Ax.T9xxx.Qxxx Kx.Txx.AQx.AKJ9x AQJ98x.QJ9xx.x.8 xxx.K8x.KJ8x.Txx 0.99696 
 P-1N-2C-X-2H-3C-P-P-P (8.0)  
 P-1N-2D-X-2H-3C-P-P-P (8.0) 
x.T8x.AT98.AQTxx KTx.AKx.Kx.KJxxx AQJ98x.QJ9xx.x.8 xxx.xx.QJxxxx.9x 0.94270 
 P-1N-2C-P-2H-3C-P-3D-P-P-P (7.0)  
 P-1N-2D-P-2H-3C-P-P-P (5.5) 
xxx.T8xx.98x.QJ9 KT.AKx.AJxx.Txxx AQJ98x.QJ9xx.x.8 xx.x.KQTxx.AKxxx 0.99830 
 P-1N-2C-X-2H-X-P-3N-P-P-P (6.5)  
 P-1N-2D-X-2H-X-P-3D-P-P-P (11.2) 
x.Tx.AT98x.K

West always has the same cards because it's your hand and it's known to you

In [4]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands_auction(auction)

for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

xxx.T8x.AK9xx.xx Tx.AKxx.Qxx.KQJx AQJ98x.QJ9xx.x.8 Kx.x.JT8x.AT9xxx 1.00
Kx.8xx.JTxxx.9xx xxx.AK.Kxx.KQTxx AQJ98x.QJ9xx.x.8 Tx.Txx.AQ98.AJxx 1.00
xx.8xx.J8xx.QJxx Kxx.Ax.AQx.KTxxx AQJ98x.QJ9xx.x.8 Tx.KTx.KT9xx.A9x 1.00
Txxx.x.T9xx.AJxx Kx.AT8xx.Kxx.KQ9 AQJ98x.QJ9xx.x.8 x.Kx.AQJ8x.Txxxx 0.45
xx.x.KT9xxx.Q9xx Kx.AKxx.Qxx.ATxx AQJ98x.QJ9xx.x.8 Txx.T8x.AJ8.KJxx 0.46
Tx.Kxxx.Qxxx.QJx Kxxx.T8.AK9x.AKx AQJ98x.QJ9xx.x.8 x.Ax.JT8x.T9xxxx 1.00
xx.ATx.9xxxx.Txx xx.Kxx.QJTx.AKQJ AQJ98x.QJ9xx.x.8 KTx.8x.AK8.9xxxx 1.00
xx.xx.T98xx.AQxx Kxx.A8x.AQx.KTxx AQJ98x.QJ9xx.x.8 Tx.KTx.KJxx.J9xx 1.00
Txx.KTx.Qxxx.Txx Kxx.Axx.JT9.AK9x AQJ98x.QJ9xx.x.8 x.8x.AK8xx.QJxxx 1.00
xx.Tx.T9xx.AK9xx Tx.AKxx.AJ8x.QJx AQJ98x.QJ9xx.x.8 Kxx.8x.KQxx.Txxx 1.00
